# Modelo de Lotka-Volterra (Predador-Presa)

O modelo de Lotka-Volterra descreve a dinâmica de dois sistemas biológicos interagindo: **presas** e **predadores**.

## Equações Diferenciais

$$\frac{dx}{dt} = \alpha x - \beta x y$$

$$\frac{dy}{dt} = \delta x y - \gamma y$$

Onde:
- $x(t)$: população de presas
- $y(t)$: população de predadores
- $\alpha$: taxa de crescimento das presas
- $\beta$: taxa de predação
- $\gamma$: taxa de mortalidade dos predadores
- $\delta$: eficiência de conversão

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact, FloatSlider, IntSlider


In [2]:
def lotka_volterra(u, t, alpha, beta, gamma, delta):
    """Modelo de Lotka-Volterra"""
    x, y = u
    dx = alpha*x - beta*x*y
    dy = delta*x*y - gamma*y
    return [dx, dy]

## Simulação Interativa

Mexa nos sliders para ver a dinâmica mudar!

In [3]:
@interact(
    alpha=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='α (cresc.)'),
    beta=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.4, description='β (pred.)'),
    gamma=FloatSlider(min=0.1, max=2.0, step=0.1, value=0.4, description='γ (mort.)'),
    delta=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.2, description='δ (efic.)'),
    x0=IntSlider(min=1, max=50, value=10, description='x₀ (presas)'),
    y0=IntSlider(min=1, max=50, value=5, description='y₀ (pred.)'),
    t_final=IntSlider(min=50, max=300, step=10, value=100, description='Tempo')
)
def plot_lotka_volterra(alpha, beta, gamma, delta, x0, y0, t_final):
    # Simulação com scipy
    t = np.linspace(0, t_final, 1000)
    u0 = [x0, y0]
    sol = odeint(lotka_volterra, u0, t, args=(alpha, beta, gamma, delta))
    
    x = sol[:, 0]
    y = sol[:, 1]
    x_eq = gamma / delta
    y_eq = alpha / beta
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Evolução temporal
    ax1.plot(t, x, 'b-', lw=2, label='Presas (x)')
    ax1.plot(t, y, 'r-', lw=2, label='Predadores (y)')
    ax1.set_xlabel('Tempo', fontsize=12)
    ax1.set_ylabel('População', fontsize=12)
    ax1.set_title('Evolução Temporal', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Diagrama de fase
    ax2.plot(x, y, 'purple', lw=2, label='Trajetória')
    ax2.plot(x0, y0, 'go', ms=10, label='Início')
    ax2.plot(x_eq, y_eq, 'r*', ms=15, label=f'Equilíbrio ({x_eq:.1f}, {y_eq:.1f})')
    ax2.set_xlabel('Presas (x)', fontsize=12)
    ax2.set_ylabel('Predadores (y)', fontsize=12)
    ax2.set_title('Diagrama de Fase', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    print(f"Ponto de equilíbrio: ({x_eq:.2f}, {y_eq:.2f})")

interactive(children=(FloatSlider(value=1.0, description='α (cresc.)', max=2.0, min=0.1), FloatSlider(value=0.…

## Análise de Estabilidade

### Pontos de Equilíbrio

1. **Trivial:** $(0, 0)$ — extinção

2. **Não-trivial:** $\left(\frac{\gamma}{\delta}, \frac{\alpha}{\beta}\right)$ — coexistência

### Matriz Jacobiana:

$$J = \begin{pmatrix} \alpha - \beta y & -\beta x \\ \delta y & \delta x - \gamma \end{pmatrix}$$

No ponto de equilíbrio:

$$J^* = \begin{pmatrix} 0 & -\frac{\beta\gamma}{\delta} \\ \frac{\delta\alpha}{\beta} & 0 \end{pmatrix}$$

### Autovalores:

$$\lambda = \pm i\sqrt{\alpha\gamma}$$

**Imaginários puros** → **centro** → órbitas fechadas

## Diferentes Condições Iniciais

In [4]:
@interact(
    alpha=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='α'),
    beta=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.4, description='β'),
    gamma=FloatSlider(min=0.1, max=2.0, step=0.1, value=0.4, description='γ'),
    delta=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.2, description='δ')
)
def plot_orbits(alpha, beta, gamma, delta):
    x_eq = gamma / delta
    y_eq = alpha / beta
    t = np.linspace(0, 150, 1000)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    ics = [(5, 2), (10, 5), (15, 8), (20, 3), (8, 12)]
    colors = ['blue', 'red', 'green', 'purple', 'orange']
    
    for (x0, y0), color in zip(ics, colors):
        sol = odeint(lotka_volterra, [x0, y0], t, args=(alpha, beta, gamma, delta))
        ax.plot(sol[:, 0], sol[:, 1], color=color, lw=2, label=f'({x0}, {y0})')
    
    ax.plot(x_eq, y_eq, 'k*', ms=20, label='Equilíbrio')
    ax.set_xlabel('Presas (x)', fontsize=12)
    ax.set_ylabel('Predadores (y)', fontsize=12)
    ax.set_title('Órbitas para diferentes condições iniciais', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='α', max=2.0, min=0.1), FloatSlider(value=0.4, descri…

## Interpretação Física

O ciclo predador-presa:

1. Muitas presas → predadores crescem
2. Muitos predadores → presas diminuem
3. Poucas presas → predadores morrem
4. Poucos predadores → presas se recuperam
5. Ciclo se repete

### Analogia: Oscilador Harmônico

O sistema é **conservativo** - existe uma integral primeira:

$$H(x,y) = \delta x - \gamma \ln(x) + \beta y - \alpha \ln(y) = \text{constante}$$

Análogo à conservação de energia mecânica.